In [60]:
import pandas as pd
import numpy as np
import seaborn as sns
import regex as re
import matplotlib.pyplot as plt

In [61]:
dramv_df = pd.read_csv('data/ALL-SAMPLES-annotations.tsv', sep='\t')
dramv_df

/Users/riddellj/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (3,4,5,6,12,13,14,18,19,20,25,26,27,34,35,36,37) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,Unnamed: 0,fasta,scaffold,gene_position,start_position,end_position,strandedness,rank,kegg_genes_id,ko_id,...,cazy_id,cazy_hits,vogdb_id,vogdb_hits,vogdb_categories,heme_regulatory_motif_count,virsorter_category,auxiliary_score,is_transposon,amg_flags
0,20100900_E1D_10_c_000000000221__full-cat_2_1,final-viral-combined-for-dramv,20100900_E1D_10_c_000000000221__full-cat_2,1,69,515,-1,C,tsa:AciPR4_0589,NaN,...,NaN,NaN,NaN,NaN,NaN,0,NaN,4,FALSE,F
1,20100900_E1D_10_c_000000000221__full-cat_2_2,final-viral-combined-for-dramv,20100900_E1D_10_c_000000000221__full-cat_2,2,716,1624,-1,C,tsa:AciPR4_0588,NaN,...,NaN,NaN,NaN,NaN,NaN,0,2,4,FALSE,F
2,20100900_E1D_10_c_000000000221__full-cat_2_3,final-viral-combined-for-dramv,20100900_E1D_10_c_000000000221__full-cat_2,3,2066,3007,1,C,ccx:COCOR_02963,NaN,...,NaN,NaN,NaN,NaN,NaN,0,1,4,FALSE,F
3,20100900_E1D_10_c_000000000221__full-cat_2_4,final-viral-combined-for-dramv,20100900_E1D_10_c_000000000221__full-cat_2,4,3154,3756,1,C,trs:Terro_2821,NaN,...,NaN,NaN,NaN,NaN,NaN,0,2,3,FALSE,F
4,20100900_E1D_10_c_000000000221__full-cat_2_5,final-viral-combined-for-dramv,20100900_E1D_10_c_000000000221__full-cat_2,5,3787,5202,1,E,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,0,NaN,3,FALSE,F
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
313673,STM_0716_E_M_E058_A.contigs_1kb_k121_994165__f...,final-viral-combined-for-dramv,STM_0716_E_M_E058_A.contigs_1kb_k121_994165__f...,5,2287,2493,1,C,cbe:Cbei_3268,NaN,...,NaN,NaN,VOG02871,REFSEQ holin; Xu,Xu,0,1.0,3,False,F
313674,STM_0716_E_M_E058_A.contigs_1kb_k121_994165__f...,final-viral-combined-for-dramv,STM_0716_E_M_E058_A.contigs_1kb_k121_994165__f...,6,2559,3323,1,C,csr:Cspa_c10340,NaN,...,NaN,NaN,NaN,NaN,NaN,0,1.0,4,False,F
313675,STM_0716_E_M_E058_A.contigs_1kb_k121_994165__f...,final-viral-combined-for-dramv,STM_0716_E_M_E058_A.contigs_1kb_k121_994165__f...,7,3485,3610,-1,E,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,0,NaN,4,False,F
313676,STM_0716_E_M_E058_A.contigs_1kb_k121_994165__f...,final-viral-combined-for-dramv,STM_0716_E_M_E058_A.contigs_1kb_k121_994165__f...,8,4120,4458,1,D,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,0,2.0,4,False,F


In [62]:
len(dramv_df)

313678

In [63]:
# remove extra header files
dramv_df = dramv_df.loc[dramv_df['fasta'] != 'fasta']
len(dramv_df)

313653

In [64]:
# assign column types to reduce memory
dramv_df = dramv_df.convert_dtypes()

In [65]:
dramv_df['auxiliary_score'] = dramv_df.auxiliary_score.astype(int)

# Filter by viral sequences in a vOTU cluster
Our dram data may still have host contamination

In [66]:
clusters_df = pd.read_csv('data/all_seqs.fna.self-blastn.clusters.tsv', header=None, sep='\t')
clusters_df.head()

,0,1
0,20160700_E14.VIRSorter_NODE_1_length_351989_co...,20160700_E14.VIRSorter_NODE_1_length_351989_co...
1,STM_0716_E_M_E034_E058_A.contigs_1kb_k121_5845...,STM_0716_E_M_E034_E058_A.contigs_1kb_k121_5845...
2,20120700_P3D.VIRSorter_NODE_161_length_217151_...,20120700_P3D.VIRSorter_NODE_161_length_217151_...
3,20130700_S2X.VIRSorter_NODE_3_length_197518_co...,20130700_S2X.VIRSorter_NODE_3_length_197518_co...
4,20140700_S11.VIRSorter_NODE_5_length_190782_co...,20140700_S11.VIRSorter_NODE_5_length_190782_co...


In [99]:
list_of_lists = [i.split(',') for i in list(clusters_df[1])]

In [100]:
contig_list = [item for sublist in list_of_lists for item in sublist]
contig_list[:10]

['20160700_E14.VIRSorter_NODE_1_length_351989_cov_12_429615_gene_124_gene_477-68188-352037-cat_5',
 'STM_0716_E_M_E034_E058_A.contigs_1kb_k121_584595||full',
 'STM_0716_E_M_E050_E054_E058_A.contigs_1kb_k121_9749814||full',
 'STM_0716_E_M_E058_A.contigs_1kb_k121_3126824||full',
 'STM_0716_E_M_E050_E054_E058_D.contigs_1kb_k121_9367843||0_partial',
 'STM_0716_E_M_E034_E058_E069_D.contigs_1kb_k121_453370||0_partial',
 '20120700_P3D.VIRSorter_NODE_161_length_217151_cov_11_826464-circular-cat_2',
 '20130700_S2X.VIRSorter_NODE_3_length_197518_cov_59_067982-cat_2',
 'STM_0716_E_M_CoAs_E069_E061_IS2_megahit_k121_433828||full',
 'STM_0716_E_M_CoAs_E069_E061_IS1_megahit_k121_5795854||full']

In [68]:
dramv_df['Unnamed: 0'].str.rsplit('_', 1, expand=True)

,0,1
0,20100900_E1D_10_c_000000000221__full-cat_2,1
1,20100900_E1D_10_c_000000000221__full-cat_2,2
2,20100900_E1D_10_c_000000000221__full-cat_2,3
3,20100900_E1D_10_c_000000000221__full-cat_2,4
4,20100900_E1D_10_c_000000000221__full-cat_2,5
...,...,...
313673,STM_0716_E_M_E058_A.contigs_1kb_k121_994165__f...,5
313674,STM_0716_E_M_E058_A.contigs_1kb_k121_994165__f...,6
313675,STM_0716_E_M_E058_A.contigs_1kb_k121_994165__f...,7
313676,STM_0716_E_M_E058_A.contigs_1kb_k121_994165__f...,8


In [69]:
dramv_df = pd.concat(
    [dramv_df,
     dramv_df['Unnamed: 0'].str.rsplit('_', 1, expand=True).rename(columns={0: 'Contig', 1: 'Gene'})
    ],
    axis=1
).reset_index()


In [75]:
dramv_df.Contig.unique()

<StringArray>
[               '20100900_E1D_10_c_000000000221__full-cat_2',
                '20100900_E1D_10_c_000000000248__full-cat_2',
                '20100900_E1D_10_c_000000000267__full-cat_2',
           '20100900_E1D_10_c_000000000306__0_partial-cat_1',
                '20100900_E1D_10_c_000000000317__full-cat_2',
                '20100900_E1D_13_c_000000004201__full-cat_2',
                '20100900_E1D_16_c_000000004546__full-cat_2',
                '20100900_E1D_16_c_000000004555__full-cat_2',
                '20100900_E1D_16_c_000000004571__full-cat_2',
                '20100900_E1D_16_c_000000004610__full-cat_2',
 ...
   'STM_0716_E_M_E058_A.contigs_1kb_k121_960787__full-cat_1',
 'STM_0716_E_M_E058_A.contigs_1kb_k121_961423__full_1-cat_1',
   'STM_0716_E_M_E058_A.contigs_1kb_k121_966691__full-cat_2',
   'STM_0716_E_M_E058_A.contigs_1kb_k121_967622__full-cat_1',
   'STM_0716_E_M_E058_A.contigs_1kb_k121_969236__full-cat_2',
   'STM_0716_E_M_E058_A.contigs_1kb_k121_974226__fu

In [106]:
dramv_df['Contig'] = dramv_df['Contig'].apply(lambda x: x.rsplit('-', 1)[0])

In [107]:
contig_list = [i.replace('||', '__') for i in contig_list]

In [108]:
clusters_dramv_df = dramv_df.loc[dramv_df['Contig'].isin(contig_list)]
clusters_dramv_df

,index,Unnamed: 0,fasta,scaffold,gene_position,start_position,end_position,strandedness,rank,kegg_genes_id,...,vogdb_id,vogdb_hits,vogdb_categories,heme_regulatory_motif_count,virsorter_category,auxiliary_score,is_transposon,amg_flags,Contig,Gene
31,31,20100900_E1D_10_c_000000000306__0_partial-cat_1_1,final-viral-combined-for-dramv,20100900_E1D_10_c_000000000306__0_partial-cat_1,1,1,1782,1,A,aba:Acid345_4404,...,<NA>,<NA>,<NA>,0,2,5,FALSE,F,20100900_E1D_10_c_000000000306__0_partial,1
32,32,20100900_E1D_10_c_000000000306__0_partial-cat_1_2,final-viral-combined-for-dramv,20100900_E1D_10_c_000000000306__0_partial-cat_1,2,1863,2096,1,E,<NA>,...,<NA>,<NA>,<NA>,0,2,4,FALSE,F,20100900_E1D_10_c_000000000306__0_partial,2
33,33,20100900_E1D_10_c_000000000306__0_partial-cat_1_3,final-viral-combined-for-dramv,20100900_E1D_10_c_000000000306__0_partial-cat_1,3,2431,3504,-1,C,aca:ACP_1178,...,<NA>,<NA>,<NA>,0,2,4,FALSE,F,20100900_E1D_10_c_000000000306__0_partial,3
34,34,20100900_E1D_10_c_000000000306__0_partial-cat_1_4,final-viral-combined-for-dramv,20100900_E1D_10_c_000000000306__0_partial-cat_1,4,3902,4225,1,E,<NA>,...,<NA>,<NA>,<NA>,0,2,4,FALSE,F,20100900_E1D_10_c_000000000306__0_partial,4
35,35,20100900_E1D_10_c_000000000306__0_partial-cat_1_5,final-viral-combined-for-dramv,20100900_E1D_10_c_000000000306__0_partial-cat_1,5,4415,4963,1,E,<NA>,...,<NA>,<NA>,<NA>,0,2,4,FALSE,F,20100900_E1D_10_c_000000000306__0_partial,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
313648,313673,STM_0716_E_M_E058_A.contigs_1kb_k121_994165__f...,final-viral-combined-for-dramv,STM_0716_E_M_E058_A.contigs_1kb_k121_994165__f...,5,2287,2493,1,C,cbe:Cbei_3268,...,VOG02871,REFSEQ holin; Xu,Xu,0,1.0,3,False,F,STM_0716_E_M_E058_A.contigs_1kb_k121_994165__full,5
313649,313674,STM_0716_E_M_E058_A.contigs_1kb_k121_994165__f...,final-viral-combined-for-dramv,STM_0716_E_M_E058_A.contigs_1kb_k121_994165__f...,6,2559,3323,1,C,csr:Cspa_c10340,...,<NA>,<NA>,<NA>,0,1.0,4,False,F,STM_0716_E_M_E058_A.contigs_1kb_k121_994165__full,6
313650,313675,STM_0716_E_M_E058_A.contigs_1kb_k121_994165__f...,final-viral-combined-for-dramv,STM_0716_E_M_E058_A.contigs_1kb_k121_994165__f...,7,3485,3610,-1,E,<NA>,...,<NA>,<NA>,<NA>,0,NaN,4,False,F,STM_0716_E_M_E058_A.contigs_1kb_k121_994165__full,7
313651,313676,STM_0716_E_M_E058_A.contigs_1kb_k121_994165__f...,final-viral-combined-for-dramv,STM_0716_E_M_E058_A.contigs_1kb_k121_994165__f...,8,4120,4458,1,D,<NA>,...,<NA>,<NA>,<NA>,0,2.0,4,False,F,STM_0716_E_M_E058_A.contigs_1kb_k121_994165__full,8


In [110]:
# define vOTU list
vOTU_list = [i.replace('||', '__') for i in list(clusters_df[0])]
vOTU_list[:10]

['20160700_E14.VIRSorter_NODE_1_length_351989_cov_12_429615_gene_124_gene_477-68188-352037-cat_5',
 'STM_0716_E_M_E034_E058_A.contigs_1kb_k121_584595__full',
 '20120700_P3D.VIRSorter_NODE_161_length_217151_cov_11_826464-circular-cat_2',
 '20130700_S2X.VIRSorter_NODE_3_length_197518_cov_59_067982-cat_2',
 '20140700_S11.VIRSorter_NODE_5_length_190782_cov_9_251422-cat_2',
 '20110700_S1S.VIRSorter_NODE_1_length_186748_cov_10_597912-cat_2',
 '20120800_E2X.VIRSorter_NODE_52_length_167441_cov_24_979359-circular-cat_2',
 '20160700_S15.VIRSorter_NODE_2_length_166797_cov_9_507389-cat_2',
 '20120800_E1X.VIRSorter_NODE_27_length_162874_cov_6_971084-cat_2',
 'Bog.Labelled.R2.T2.F1.trimmomatic.metaspades_bin.69_84.77_5.37_NODE_16_length_161907_cov_18.638831__full']

In [111]:
vOTU_dramv_df = clusters_dramv_df.loc[clusters_dramv_df['Contig'].isin(vOTU_list)]
vOTU_dramv_df

,index,Unnamed: 0,fasta,scaffold,gene_position,start_position,end_position,strandedness,rank,kegg_genes_id,...,vogdb_id,vogdb_hits,vogdb_categories,heme_regulatory_motif_count,virsorter_category,auxiliary_score,is_transposon,amg_flags,Contig,Gene
31,31,20100900_E1D_10_c_000000000306__0_partial-cat_1_1,final-viral-combined-for-dramv,20100900_E1D_10_c_000000000306__0_partial-cat_1,1,1,1782,1,A,aba:Acid345_4404,...,<NA>,<NA>,<NA>,0,2,5,FALSE,F,20100900_E1D_10_c_000000000306__0_partial,1
32,32,20100900_E1D_10_c_000000000306__0_partial-cat_1_2,final-viral-combined-for-dramv,20100900_E1D_10_c_000000000306__0_partial-cat_1,2,1863,2096,1,E,<NA>,...,<NA>,<NA>,<NA>,0,2,4,FALSE,F,20100900_E1D_10_c_000000000306__0_partial,2
33,33,20100900_E1D_10_c_000000000306__0_partial-cat_1_3,final-viral-combined-for-dramv,20100900_E1D_10_c_000000000306__0_partial-cat_1,3,2431,3504,-1,C,aca:ACP_1178,...,<NA>,<NA>,<NA>,0,2,4,FALSE,F,20100900_E1D_10_c_000000000306__0_partial,3
34,34,20100900_E1D_10_c_000000000306__0_partial-cat_1_4,final-viral-combined-for-dramv,20100900_E1D_10_c_000000000306__0_partial-cat_1,4,3902,4225,1,E,<NA>,...,<NA>,<NA>,<NA>,0,2,4,FALSE,F,20100900_E1D_10_c_000000000306__0_partial,4
35,35,20100900_E1D_10_c_000000000306__0_partial-cat_1_5,final-viral-combined-for-dramv,20100900_E1D_10_c_000000000306__0_partial-cat_1,5,4415,4963,1,E,<NA>,...,<NA>,<NA>,<NA>,0,2,4,FALSE,F,20100900_E1D_10_c_000000000306__0_partial,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
313532,313557,STM_0716_E_M_E058_A.contigs_1kb_k121_960787__f...,final-viral-combined-for-dramv,STM_0716_E_M_E058_A.contigs_1kb_k121_960787__f...,51,42678,43883,1,C,csr:Cspa_c27090,...,<NA>,<NA>,<NA>,0,1.0,2,False,TF,STM_0716_E_M_E058_A.contigs_1kb_k121_960787__full,51
313533,313558,STM_0716_E_M_E058_A.contigs_1kb_k121_960787__f...,final-viral-combined-for-dramv,STM_0716_E_M_E058_A.contigs_1kb_k121_960787__f...,52,43898,44194,1,C,csb:CLSA_c22930,...,<NA>,<NA>,<NA>,0,1.0,2,False,TF,STM_0716_E_M_E058_A.contigs_1kb_k121_960787__full,52
313534,313559,STM_0716_E_M_E058_A.contigs_1kb_k121_960787__f...,final-viral-combined-for-dramv,STM_0716_E_M_E058_A.contigs_1kb_k121_960787__f...,53,44271,45473,1,D,<NA>,...,<NA>,<NA>,<NA>,0,2.0,2,False,MTF,STM_0716_E_M_E058_A.contigs_1kb_k121_960787__full,53
313535,313560,STM_0716_E_M_E058_A.contigs_1kb_k121_960787__f...,final-viral-combined-for-dramv,STM_0716_E_M_E058_A.contigs_1kb_k121_960787__f...,54,45473,46312,1,E,<NA>,...,<NA>,<NA>,<NA>,0,2.0,2,False,TF,STM_0716_E_M_E058_A.contigs_1kb_k121_960787__full,54


In [112]:
vOTU_dramv_df.Contig.nunique()

5622

# Permissive AMG identification

In [7]:
dramv_df.auxiliary_score.value_counts()

4    129055
5     59284
2     50564
3     44150
1     30600
Name: auxiliary_score, dtype: int64

In [42]:
permissive_df = (
    dramv_df
    .loc[
        (dramv_df['amg_flags'].str.contains('M')) & # contains metabolism flag
        (dramv_df['auxiliary_score'] <= 3) & # auxiliary score <= 3, meaning flanked by virus-like or viral marker genes
        ~(dramv_df['amg_flags'].str.contains('T')) # does not contain transposon flag
    ]
)
len(permissive_df)

4084

In [10]:
permissive_df.amg_flags.value_counts()

MKF      1436
MF       1093
MK        713
M         451
MKPF       72
MKEF       53
VMF        44
MPF        44
MKP        43
VM         31
MKE        28
MKA        19
MKAF       16
MP         10
VMKEF      10
VMKE        9
VMKF        3
MAF         3
MA          2
VMK         2
MKAP        1
MKAPF       1
Name: amg_flags, dtype: Int64

In [11]:
# what do they do?
permissive_df

,Unnamed: 0,fasta,scaffold,gene_position,start_position,end_position,strandedness,rank,kegg_genes_id,ko_id,...,cazy_id,cazy_hits,vogdb_id,vogdb_hits,vogdb_categories,heme_regulatory_motif_count,virsorter_category,auxiliary_score,is_transposon,amg_flags
90,20100900_E1D_16_c_000000004571__full-cat_2_5,final-viral-combined-for-dramv,20100900_E1D_16_c_000000004571__full-cat_2,5,4709,6559,1,A,ial:IALB_1740,<NA>,...,GH13; GH13_10; GH13_12; GH13_13; GH13_14; GH13...,GH13 alpha-amylase (EC 3.2.1.1); pullulanase (...,<NA>,<NA>,<NA>,0,2,3,FALSE,MF
175,20100900_E1D_9_c_000000003104__full-cat_2_5,final-viral-combined-for-dramv,20100900_E1D_9_c_000000003104__full-cat_2,5,2962,3561,-1,C,pms:KNP414_04182,K00969,...,<NA>,<NA>,<NA>,<NA>,<NA>,0,2,3,FALSE,MKF
200,20100900_E2D_7_c_000000000804__full-cat_2_5,final-viral-combined-for-dramv,20100900_E2D_7_c_000000000804__full-cat_2,5,4608,5642,-1,A,scl:sce2946,K02040,...,<NA>,<NA>,<NA>,<NA>,<NA>,0,1,3,FALSE,MKEF
202,20100900_E2D_7_c_000000000804__full-cat_2_7,final-viral-combined-for-dramv,20100900_E2D_7_c_000000000804__full-cat_2,7,6261,7157,1,C,mro:MROS_0061,<NA>,...,<NA>,<NA>,<NA>,<NA>,<NA>,0,2,3,FALSE,MKF
278,20100900_E2D_7_c_000000000943__full-cat_2_10,final-viral-combined-for-dramv,20100900_E2D_7_c_000000000943__full-cat_2,10,13825,15321,-1,A,mro:MROS_2369,<NA>,...,<NA>,<NA>,<NA>,<NA>,<NA>,0,2,3,FALSE,MKF
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
313332,STM_0716_E_M_E058_A.contigs_1kb_k121_9348__ful...,final-viral-combined-for-dramv,STM_0716_E_M_E058_A.contigs_1kb_k121_9348__ful...,8,3248,4003,1,C,bst:GYO_1077,K00627,...,<NA>,<NA>,<NA>,<NA>,<NA>,0,2.0,2,False,MF
313385,STM_0716_E_M_E058_A.contigs_1kb_k121_937465__f...,final-viral-combined-for-dramv,STM_0716_E_M_E058_A.contigs_1kb_k121_937465__f...,5,1695,2237,1,C,cbn:CbC4_4053,K00942,...,<NA>,<NA>,VOG00939,sp|Q16774|KGUA_HUMAN Guanylate kinase; Xh,Xh,0,1.0,3,False,MKF
313436,STM_0716_E_M_E058_A.contigs_1kb_k121_950472__f...,final-viral-combined-for-dramv,STM_0716_E_M_E058_A.contigs_1kb_k121_950472__f...,13,8513,9421,1,C,mai:MICA_75,K00945,...,<NA>,<NA>,<NA>,<NA>,<NA>,0,2.0,3,False,MF
313467,STM_0716_E_M_E058_A.contigs_1kb_k121_951508__f...,final-viral-combined-for-dramv,STM_0716_E_M_E058_A.contigs_1kb_k121_951508__f...,29,26271,27245,1,A,hif:HIBPF07341,K00558,...,<NA>,<NA>,VOG01537,sp|P34877|MTSA_LACLC Type II methyltransferase...,Xh;Xp,0,2.0,2,False,M


In [12]:
# filter for only AMGs found in our viruses of interest
simplified_permissive_df = permissive_df[['scaffold', 'kegg_hit', 'ko_id', 'viral_hit', 'peptidase_hit', 'cazy_hits', 'vogdb_hits', 'pfam_hits', 'amg_flags']]

In [13]:
simplified_permissive_df.loc[simplified_permissive_df['kegg_hit'].str.contains('met')]

,scaffold,kegg_hit,ko_id,viral_hit,peptidase_hit,cazy_hits,vogdb_hits,pfam_hits,amg_flags
352,20100900_E2D_7_c_000000001071__full-cat_2,rpc:RPC_4309 DNA-cytosine methyltransferase; ...,K00558,YP_008059411.1 AdoMet-MTase [Halovirus HGTV-1],<NA>,<NA>,sp|P34877|MTSA_LACLC Type II methyltransferase...,C-5 cytosine-specific DNA methylase [PF00145.20],MF
1146,20110600_P1M_6_c_000000005038__0_partial-cat_1,mpo:Mpop_2731 C-5 cytosine-specific DNA methy...,K00558,YP_009785467.1 methyltransferase [Caulobacter ...,<NA>,<NA>,sp|P34877|MTSA_LACLC Type II methyltransferase...,C-5 cytosine-specific DNA methylase [PF00145.20],M
4772,20110700_P3D_1_c_000000001257__full-cat_2,sus:Acid_7194 methionine aminopeptidase (EC:3...,K01265,<NA>,MER0051252 - methionyl aminopeptidase 1 ({Stap...,<NA>,<NA>,Metallopeptidase family M24 [PF00557.27],MF
6084,20110700_S3M_1_c_000000001866__full-cat_1,fli:Fleli_1473 glycine/serine hydroxymethyltr...,<NA>,YP_004956814.1 orf66 gene product [Helicoverpa...,<NA>,<NA>,sp|A6Q478|GLYA_NITSB Serine hydroxymethyltrans...,Serine hydroxymethyltransferase [PF00464.22],MK
6087,20110700_S3M_1_c_000000001866__full-cat_1,lan:Lacal_0970 methylmalonyl-CoA epimerase; K...,K05606,<NA>,<NA>,<NA>,<NA>,Glyoxalase/Bleomycin resistance protein/Dioxyg...,M
...,...,...,...,...,...,...,...,...,...
309521,STM_0716_E_M_E058_A.contigs_1kb_k121_2716883__...,pdi:BDI_3694 protoporphyrinogen oxidase; K024...,K02493,<NA>,<NA>,<NA>,<NA>,Methyltransferase small domain [PF05175.17]; M...,MK
310941,STM_0716_E_M_E058_A.contigs_1kb_k121_3126810__...,lmj:LMOG_03106 hypothetical protein; K00558 D...,K00558,YP_009097731.1 methyltransferase [Citrobacter ...,<NA>,<NA>,sp|P34877|MTSA_LACLC Type II methyltransferase...,C-5 cytosine-specific DNA methylase [PF00145.20],M
311975,STM_0716_E_M_E058_A.contigs_1kb_k121_453867__f...,cbh:CLC_0945 DNA-cytosine methyltransferase; ...,K00558,YP_009626360.1 DNA-cytosine methyltransferase ...,<NA>,<NA>,sp|P34877|MTSA_LACLC Type II methyltransferase...,C-5 cytosine-specific DNA methylase [PF00145.20],MF
312669,STM_0716_E_M_E058_A.contigs_1kb_k121_686233__f...,cle:Clole_1843 DNA-cytosine methyltransferase...,K00558,YP_009985271.1 cytosine methyltransferase [Esc...,<NA>,<NA>,<NA>,C-5 cytosine-specific DNA methylase [PF00145.20],MF


In [14]:
# get list of 'met' related AMGs
simplified_permissive_df.loc[simplified_permissive_df['pfam_hits'].str.contains('met')]['pfam_hits'].unique()

<StringArray>
[                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    'C-5 cytosine-specific DNA methylase [PF00145.20]',
                                                                                                                                                                                                                                                 

In [19]:
pfam_methane_metabolism = ['PF02249', 'PF02745', 'PF02461', 'PF02332']

In [25]:
def get_methane_metabolism(s):
    for i in pfam_methane_metabolism:
        if i in s:
            return True
    return False

In [37]:
# get list of AMGs doing methane metabolism
simplified_permissive_df['has_pfam_methane_metabolism'] = simplified_permissive_df['pfam_hits'].fillna(' ').apply(get_methane_metabolism)

<ipython-input-37-fc10eff5ef35>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  simplified_permissive_df['has_pfam_methane_metabolism'] = simplified_permissive_df['pfam_hits'].fillna(' ').apply(get_methane_metabolism)


In [39]:
any(list(simplified_permissive_df['has_pfam_methane_metabolism']))

False

In [33]:
simplified_permissive_df.loc[simplified_permissive_df['has_pfam_methane_metabolism'] == True]

,scaffold,kegg_hit,ko_id,viral_hit,peptidase_hit,cazy_hits,vogdb_hits,pfam_hits,amg_flags,has_pfam_methane_metabolism


In [113]:
# look at vOTU genes
vOTU_dramv_df

,index,Unnamed: 0,fasta,scaffold,gene_position,start_position,end_position,strandedness,rank,kegg_genes_id,...,vogdb_id,vogdb_hits,vogdb_categories,heme_regulatory_motif_count,virsorter_category,auxiliary_score,is_transposon,amg_flags,Contig,Gene
31,31,20100900_E1D_10_c_000000000306__0_partial-cat_1_1,final-viral-combined-for-dramv,20100900_E1D_10_c_000000000306__0_partial-cat_1,1,1,1782,1,A,aba:Acid345_4404,...,<NA>,<NA>,<NA>,0,2,5,FALSE,F,20100900_E1D_10_c_000000000306__0_partial,1
32,32,20100900_E1D_10_c_000000000306__0_partial-cat_1_2,final-viral-combined-for-dramv,20100900_E1D_10_c_000000000306__0_partial-cat_1,2,1863,2096,1,E,<NA>,...,<NA>,<NA>,<NA>,0,2,4,FALSE,F,20100900_E1D_10_c_000000000306__0_partial,2
33,33,20100900_E1D_10_c_000000000306__0_partial-cat_1_3,final-viral-combined-for-dramv,20100900_E1D_10_c_000000000306__0_partial-cat_1,3,2431,3504,-1,C,aca:ACP_1178,...,<NA>,<NA>,<NA>,0,2,4,FALSE,F,20100900_E1D_10_c_000000000306__0_partial,3
34,34,20100900_E1D_10_c_000000000306__0_partial-cat_1_4,final-viral-combined-for-dramv,20100900_E1D_10_c_000000000306__0_partial-cat_1,4,3902,4225,1,E,<NA>,...,<NA>,<NA>,<NA>,0,2,4,FALSE,F,20100900_E1D_10_c_000000000306__0_partial,4
35,35,20100900_E1D_10_c_000000000306__0_partial-cat_1_5,final-viral-combined-for-dramv,20100900_E1D_10_c_000000000306__0_partial-cat_1,5,4415,4963,1,E,<NA>,...,<NA>,<NA>,<NA>,0,2,4,FALSE,F,20100900_E1D_10_c_000000000306__0_partial,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
313532,313557,STM_0716_E_M_E058_A.contigs_1kb_k121_960787__f...,final-viral-combined-for-dramv,STM_0716_E_M_E058_A.contigs_1kb_k121_960787__f...,51,42678,43883,1,C,csr:Cspa_c27090,...,<NA>,<NA>,<NA>,0,1.0,2,False,TF,STM_0716_E_M_E058_A.contigs_1kb_k121_960787__full,51
313533,313558,STM_0716_E_M_E058_A.contigs_1kb_k121_960787__f...,final-viral-combined-for-dramv,STM_0716_E_M_E058_A.contigs_1kb_k121_960787__f...,52,43898,44194,1,C,csb:CLSA_c22930,...,<NA>,<NA>,<NA>,0,1.0,2,False,TF,STM_0716_E_M_E058_A.contigs_1kb_k121_960787__full,52
313534,313559,STM_0716_E_M_E058_A.contigs_1kb_k121_960787__f...,final-viral-combined-for-dramv,STM_0716_E_M_E058_A.contigs_1kb_k121_960787__f...,53,44271,45473,1,D,<NA>,...,<NA>,<NA>,<NA>,0,2.0,2,False,MTF,STM_0716_E_M_E058_A.contigs_1kb_k121_960787__full,53
313535,313560,STM_0716_E_M_E058_A.contigs_1kb_k121_960787__f...,final-viral-combined-for-dramv,STM_0716_E_M_E058_A.contigs_1kb_k121_960787__f...,54,45473,46312,1,E,<NA>,...,<NA>,<NA>,<NA>,0,2.0,2,False,TF,STM_0716_E_M_E058_A.contigs_1kb_k121_960787__full,54


In [116]:
vOTU_permissive_df = (
    vOTU_dramv_df
    .loc[
        (vOTU_dramv_df['amg_flags'].str.contains('M')) & # contains metabolism flag
        (vOTU_dramv_df['auxiliary_score'] <= 3) & # auxiliary score <= 3, meaning flanked by virus-like or viral marker genes
        ~(vOTU_dramv_df['amg_flags'].str.contains('T')) # does not contain transposon flag
    ]
)

In [117]:
# filter for only AMGs found in our viruses of interest
simplified_vOTU_permissive_df = vOTU_permissive_df[['scaffold', 'kegg_hit', 'ko_id', 'viral_hit', 'peptidase_hit', 'cazy_hits', 'vogdb_hits', 'pfam_hits', 'amg_flags']]

In [118]:
simplified_vOTU_permissive_df

,scaffold,kegg_hit,ko_id,viral_hit,peptidase_hit,cazy_hits,vogdb_hits,pfam_hits,amg_flags
655,20110600_P1D_5_c_000000003835__full-cat_1,say:TPY_1071 fbaB; Fructose-bisphosphate aldo...,K01623,<NA>,<NA>,<NA>,<NA>,DeoC/LacD family aldolase [PF01791.12],MKF
657,20110600_P1D_5_c_000000003835__full-cat_1,hau:Haur_1907 3-hydroxyisobutyrate dehydrogen...,K00020,<NA>,<NA>,<NA>,<NA>,NAD binding domain of 6-phosphogluconate dehyd...,MKF
2329,20110600_S3M_7_c_000000000076__full-cat_1,bcj:BCAM1078 putative phage glucosaminidase,<NA>,<NA>,<NA>,GH73 lysozyme (EC 3.2.1.17); mannosyl-glycopro...,sp|Q9I4P4|FLGJ_PSEAE Peptidoglycan hydrolase F...,Mannosyl-glycoprotein endo-beta-N-acetylglucos...,MKAF
2362,20110600_S3M_7_c_000000000283__full-cat_2,sus:Acid_3844 galactokinase (EC:2.7.1.6); K00...,K00849,<NA>,<NA>,<NA>,<NA>,Galactokinase galactose-binding signature [PF1...,MK
2363,20110600_S3M_7_c_000000000283__full-cat_2,sus:Acid_4589 ABC transporter-like protein; K...,K02065,YP_009001498.1 putative ATP-binding cassette t...,<NA>,<NA>,sp|P44662|Y361_HAEIN Probable iron transport s...,ABC transporter [PF00005.30],MF
...,...,...,...,...,...,...,...,...,...
309240,STM_0716_E_M_E058_A.contigs_1kb_k121_2672491__...,tmb:Thimo_0605 hypothetical protein,<NA>,<NA>,<NA>,<NA>,<NA>,Concanavalin A-like lectin/glucanases superfam...,MKF
310916,STM_0716_E_M_E058_A.contigs_1kb_k121_3126810__...,lmj:LMOG_03106 hypothetical protein; K00558 D...,K00558,YP_009097731.1 methyltransferase [Citrobacter ...,<NA>,<NA>,sp|P34877|MTSA_LACLC Type II methyltransferase...,C-5 cytosine-specific DNA methylase [PF00145.20],M
311761,STM_0716_E_M_E058_A.contigs_1kb_k121_398302__f...,aba:Acid345_4695 nuclease-related protein,<NA>,<NA>,<NA>,<NA>,<NA>,PLD-like domain [PF13091.9],MK
311950,STM_0716_E_M_E058_A.contigs_1kb_k121_453867__f...,cbh:CLC_0945 DNA-cytosine methyltransferase; ...,K00558,YP_009626360.1 DNA-cytosine methyltransferase ...,<NA>,<NA>,sp|P34877|MTSA_LACLC Type II methyltransferase...,C-5 cytosine-specific DNA methylase [PF00145.20],MF


In [119]:
clusters_permissive_df = (
    clusters_dramv_df
    .loc[
        (clusters_dramv_df['amg_flags'].str.contains('M')) & # contains metabolism flag
        (clusters_dramv_df['auxiliary_score'] <= 3) & # auxiliary score <= 3, meaning flanked by virus-like or viral marker genes
        ~(clusters_dramv_df['amg_flags'].str.contains('T')) # does not contain transposon flag
    ]
)

In [120]:
clusters_permissive_df

,index,Unnamed: 0,fasta,scaffold,gene_position,start_position,end_position,strandedness,rank,kegg_genes_id,...,vogdb_id,vogdb_hits,vogdb_categories,heme_regulatory_motif_count,virsorter_category,auxiliary_score,is_transposon,amg_flags,Contig,Gene
655,655,20110600_P1D_5_c_000000003835__full-cat_1_9,final-viral-combined-for-dramv,20110600_P1D_5_c_000000003835__full-cat_1,9,8015,8920,-1,C,say:TPY_1071,...,<NA>,<NA>,<NA>,0,2,2,FALSE,MKF,20110600_P1D_5_c_000000003835__full,9
657,657,20110600_P1D_5_c_000000003835__full-cat_1_11,final-viral-combined-for-dramv,20110600_P1D_5_c_000000003835__full-cat_1,11,10072,11004,1,C,hau:Haur_1907,...,<NA>,<NA>,<NA>,0,2,2,FALSE,MKF,20110600_P1D_5_c_000000003835__full,11
1146,1146,20110600_P1M_6_c_000000005038__0_partial-cat_1_15,final-viral-combined-for-dramv,20110600_P1M_6_c_000000005038__0_partial-cat_1,15,6969,8426,1,C,mpo:Mpop_2731,...,VOG01537,sp|P34877|MTSA_LACLC Type II methyltransferase...,Xh;Xp,0,1,2,FALSE,M,20110600_P1M_6_c_000000005038__0_partial,15
1190,1190,20110600_P1M_6_c_000000005038__0_partial-cat_1_59,final-viral-combined-for-dramv,20110600_P1M_6_c_000000005038__0_partial-cat_1,59,34932,37598,1,C,kfl:Kfla_2748,...,<NA>,<NA>,<NA>,0,2,1,FALSE,MKF,20110600_P1M_6_c_000000005038__0_partial,59
2329,2329,20110600_S3M_7_c_000000000076__full-cat_1_10,final-viral-combined-for-dramv,20110600_S3M_7_c_000000000076__full-cat_1,10,4950,5444,-1,C,bcj:BCAM1078,...,VOG01377,sp|Q9I4P4|FLGJ_PSEAE Peptidoglycan hydrolase F...,Xh,0,1,2,FALSE,MKAF,20110600_S3M_7_c_000000000076__full,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
313276,313301,STM_0716_E_M_E058_A.contigs_1kb_k121_905594__f...,final-viral-combined-for-dramv,STM_0716_E_M_E058_A.contigs_1kb_k121_905594__f...,8,7130,9670,1,A,ccb:Clocel_1176,...,<NA>,<NA>,<NA>,0,2.0,3,False,M,STM_0716_E_M_E058_A.contigs_1kb_k121_905594__full,8
313307,313332,STM_0716_E_M_E058_A.contigs_1kb_k121_9348__ful...,final-viral-combined-for-dramv,STM_0716_E_M_E058_A.contigs_1kb_k121_9348__ful...,8,3248,4003,1,C,bst:GYO_1077,...,<NA>,<NA>,<NA>,0,2.0,2,False,MF,STM_0716_E_M_E058_A.contigs_1kb_k121_9348__full,8
313360,313385,STM_0716_E_M_E058_A.contigs_1kb_k121_937465__f...,final-viral-combined-for-dramv,STM_0716_E_M_E058_A.contigs_1kb_k121_937465__f...,5,1695,2237,1,C,cbn:CbC4_4053,...,VOG00939,sp|Q16774|KGUA_HUMAN Guanylate kinase; Xh,Xh,0,1.0,3,False,MKF,STM_0716_E_M_E058_A.contigs_1kb_k121_937465__full,5
313442,313467,STM_0716_E_M_E058_A.contigs_1kb_k121_951508__f...,final-viral-combined-for-dramv,STM_0716_E_M_E058_A.contigs_1kb_k121_951508__f...,29,26271,27245,1,A,hif:HIBPF07341,...,VOG01537,sp|P34877|MTSA_LACLC Type II methyltransferase...,Xh;Xp,0,2.0,2,False,M,STM_0716_E_M_E058_A.contigs_1kb_k121_951508__full,29


In [121]:
simplified_vOTU_permissive_df.pfam_hits.value_counts()

C-5 cytosine-specific DNA methylase [PF00145.20]                                                                                  52
Glycosyl transferases group 1 [PF00534.23]; Glycosyl transferases group 1 [PF13692.9]                                             35
Concanavalin A-like lectin/glucanases superfamily [PF13385.9]                                                                     25
Mannosyl-glycoprotein endo-beta-N-acetylglucosaminidase [PF01832.23]                                                              14
ABC transporter [PF00005.30]                                                                                                       9
                                                                                                                                  ..
Pyruvate formate lyase-like [PF02901.18]                                                                                           1
Alpha-L-arabinofuranosidase C-terminal domain [PF06964.15]; 3-keto-di